# Fire Perimeter Exploration

This notebook attempts to demonstrate how large cloud datasets relevant to the disaster response community can be processed and analyzed in a Jupyter notebook.

This notebook explores near real-time fire affected areas from active wildfires from the [VIIRS Modeled Fire Perimiters dataset](https://firms.modaps.eosdis.nasa.gov/descriptions/FEDS_VIIRS_SNPP), which is updated every 12 hours and fetched from the VEDA 

This data is fetched from the [VEDA TiPG API](https://openveda.cloud/api/features/), which is the source of the [VEDA Fire Event Explorer](https://www.earthdata.nasa.gov/dashboard/tools/fire-event-explorer).

This fire extent data is paired with the [Microsoft Building Footprints](https://planetarycomputer.microsoft.com/dataset/ms-buildings#overview) dataset, a collection of almost 1 billion polygons of buildings around the world.

The notebook focuses on the US state of New Mexico, but can be adapted for other regions.

## Import Libraries

In [17]:
import requests
from owslib.ogcapi.features import Features
from pystac_client import Client

import obstore as obs
from obstore.store import from_url, AzureStore
from obstore.auth.planetary_computer import PlanetaryComputerCredentialProvider
import planetary_computer

import shapely.geometry
import mercantile

import pyarrow.parquet as pq

import deltalake
import adlfs

from lonboard import viz
import pandas as pd
import geopandas as gpd
import jupytergis

import rasterio as rs
from rasterio.plot import show

## Define our AOI

We'll be exploring data in the US state of New Mexico.

In [126]:
aoi_polygon = shapely.geometry.box("-110", "31", "-103", "38") 
aoi = ["-110", "31", "-103", "38"]

## Get Fire Perimeter data

The current vector data is hosted as a [TiPG](https://developmentseed.org/tipg/) feature (and tile) API, which is distinct from the [VEDA STAC API](https://openveda.cloud/).

This section is similar to [this notebook](https://docs.openveda.cloud/user-guide/notebooks/tutorials/mapping-fires.html).

In [57]:
VEDA_TiPG_url = "https://openveda.cloud/api/features/"

Datasets distributed through OGC APIs are organized into collections. We can see the available collections with this command:

In [58]:
w = Features(VEDA_TiPG_url)
w.feature_collections()

['public.eis_fire_lf_perimeter_archive',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_snapshot_perimeter_nrt',
 'pg_temp.eis_fire_lf_perimeter_nrt_latest',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

The `public.eis_fire_snapshot_perimeter_nrt` collection shows the extent of active fires. We will use that as our collection.

In [ ]:
perim = w.collection("public.eis_fire_snapshot_perimeter_nrt")

We can look at the variables, or queryables, that the collection has available:

In [124]:
perim_q = w.collection_queryables("public.eis_fire_snapshot_perimeter_nrt")
list(perim_q["properties"])

['geometry',
 'duration',
 'farea',
 'fireid',
 'flinelen',
 'fperim',
 'geom_counts',
 'isactive',
 'low_confidence_grouping',
 'meanfrp',
 'n_newpixels',
 'n_pixels',
 'pixden',
 'primarykey',
 'region',
 't']

We can get the items within the collection that are within our AOI, and add them to a GeoDataFrame.

In [141]:
perim_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=aoi,
    limit=1000
)

perimeters = gpd.GeoDataFrame.from_features(perim_results["features"], crs="EPSG:4326")
print("There are", len(perimeters), "active wildfire perimeters within the AOI.")

There are 93 active wildfire perimeters within the AOI.


We can quickly visualize these active wildfire perimeters with [Lonboard's](https://developmentseed.org/lonboard/latest/) `viz` function. Lonboard excels at visualizing large datasets, and the `viz` function lets us do so quickly.

In [142]:
viz(perimeters)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Buildings

In [83]:
mpc_url = "https://planetarycomputer.microsoft.com/api/stac/v1/"
mpc = Client.open(mpc_url)
mpc.title

'Microsoft Planetary Computer STAC API'

In [84]:
mpc_search = mpc.collection_search(
    q="building"
)

print(f"{mpc_search.matched()} collections found")

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


2 collections found


In [85]:
for collection in mpc_search.collections():
    print(collection.id)

3dep-lidar-classification
ms-buildings


In [86]:
buildings_cat = mpc.get_collection("ms-buildings")
buildings_cat.title

'Microsoft Building Footprints'

In [87]:
building_regions_search = mpc.search(
    collections=["ms-buildings"],
    bbox=new_mexico
).item_collection_as_dict()

In [88]:
building_regions = gpd.GeoDataFrame.from_features(building_regions_search["features"])
building_regions

,geometry,title,datetime,description,table:columns,table:row_count,msbuildings:region,end_datetime,start_datetime
0,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-07-06T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",129554779,United States,NaN,NaN
1,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-06-14T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",129557523,United States,NaN,NaN
2,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
3,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
4,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00
5,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00


In [139]:
search = mpc.search(
    collections=["ms-buildings"],
    intersects=new_mexico_polygon,
    query={
    # "msbuildings:region": {"eq": "United States"},
    # "msbuildings:processing-date": {"eq": "2023-04-25"}
    },
)

ic = search.item_collection()

In [140]:
list(ic)

[<Item id=United States_2022-07-06>,
 <Item id=United States_2022-06-14>,
 <Item id=Mexico_2022-07-06>,
 <Item id=Mexico_2022-06-14>,
 <Item id=North America_2022-07-06>,
 <Item id=North America_2022-06-14>]

In [152]:
ic[0]

<Item id=United States_2022-07-06>

In [133]:
parquet_file_info = ic[0].assets["data"]

In [134]:
parquet_file_info

<Asset href=abfs://footprints/global/2022-07-06/ml-buildings.parquet/RegionName=United States>

In [135]:
authenticate_request = requests.get(url=f"https://planetarycomputer.microsoft.com/api/sas/v1/token/{buildings_cat.id}")
token = authenticate_request.json()["token"]
storage_opts = parquet_file_info.extra_fields["table:storage_options"]

In [141]:
store = AzureStore(credential_provider=PlanetaryComputerCredentialProvider(parquet_file_info.href, account_name=storage_opts["account_name"]))

In [154]:
files = obs.list(store, return_arrow=True)

In [3]:
quadkeys = [
    int(mercantile.quadkey(tile))
    for tile in mercantile.tiles(*new_mexico_polygon.bounds, zooms=9)
]
quadkeys

[23100213,
 23100231,
 23100233,
 23102011,
 23102013,
 23102031,
 23102033,
 23102211,
 23102213,
 23102231,
 23102233,
 23120011,
 23120013,
 23100302,
 23100320,
 23100322,
 23102100,
 23102102,
 23102120,
 23102122,
 23102300,
 23102302,
 23102320,
 23102322,
 23120100,
 23120102,
 23100303,
 23100321,
 23100323,
 23102101,
 23102103,
 23102121,
 23102123,
 23102301,
 23102303,
 23102321,
 23102323,
 23120101,
 23120103,
 23100312,
 23100330,
 23100332,
 23102110,
 23102112,
 23102130,
 23102132,
 23102310,
 23102312,
 23102330,
 23102332,
 23120110,
 23120112,
 23100313,
 23100331,
 23100333,
 23102111,
 23102113,
 23102131,
 23102133,
 23102311,
 23102313,
 23102331,
 23102333,
 23120111,
 23120113,
 23101202,
 23101220,
 23101222,
 23103000,
 23103002,
 23103020,
 23103022,
 23103200,
 23103202,
 23103220,
 23103222,
 23121000,
 23121002,
 23101203,
 23101221,
 23101223,
 23103001,
 23103003,
 23103021,
 23103023,
 23103201,
 23103203,
 23103221,
 23103223,
 23121001,
 23121003,

In [47]:
fire_quadkeys = set()
for geometry in perimeters.geometry:
    quadkeys = [
        int(mercantile.quadkey(tile))
        for tile in mercantile.tiles(*geometry.bounds, zooms=9)
    ]
    fire_quadkeys.update(quadkeys)

fire_quadkeys = list(fire_quadkeys)

In [ ]:
parquet_urls = [f for f in files if any(f"quadkey={qk}" in f for qk in quadkeys)]
parquet_urls

NameError: name 'files' is not defined

In [130]:
path = "part-00821-2abf88bf-ebfb-42fa-9843-5951065b6382.c000.snappy.parquet"

parquet_test = store.get(path).bytes()

with open("parquet_test.parquet", "wb") as f:
    f.write(parquet_test)

In [131]:
test_gpd = gpd.read_parquet("parquet_test.parquet")
test_gpd

,geometry
0,"POLYGON ((-100.00058 48.37437, -100.00058 48.3..."
1,"POLYGON ((-101.80731 33.56686, -101.80724 33.5..."
2,"POLYGON ((-102.38294 31.86974, -102.38291 31.8..."
3,"POLYGON ((-104.80376 39.35458, -104.80388 39.3..."
4,"POLYGON ((-100.4272 48.61329, -100.4272 48.613..."
...,...
649637,"POLYGON ((-93.33224 44.79579, -93.33223 44.795..."
649638,"POLYGON ((-95.438 29.92417, -95.43799 29.92424..."
649639,"POLYGON ((-96.18249 41.21011, -96.18245 41.210..."
649640,"POLYGON ((-97.39348 28.67374, -97.39361 28.673..."


In [132]:
viz(test_gpd)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

### Try deltalake

In [49]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

In [50]:
asset = collection.assets["delta"]

In [51]:
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}

In [52]:
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

In [53]:
file_uris = table.file_uris([("RegionName", "=", "UnitedStates"),("quadkey", "in", fire_quadkeys)])
file_uris

['az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102312/part-00005-b32140c2-71a7-4634-ad9b-e468f8e82e70.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102330/part-00005-c55f6fcd-acbc-4139-b202-de34f44a596c.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103322/part-00033-93412344-ba19-4f70-a6bb-d89e38509ca6.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103120/part-00051-e6d7e01e-4c98-497e-aa48-314143b640ce.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23101221/part-00177-0c9497f0-7bdc-4753-ab03-6dc144361eaa.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102231/part-00182-ad1c97ca-065f-450c-b53c-98c600d900d8.c000.snappy.parquet'

In [54]:
gdf = pd.concat(
    [
        gpd.read_parquet(file_uri, storage_options=storage_options)
        for file_uri in file_uris
    ]
)

In [96]:
metadata_geojson = gpd.GeoDataFrame.from_file("https://minedbuildings.z5.web.core.windows.net/global-buildings/buildings-coverage.geojson")

In [98]:
viz(metadata_geojson)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [55]:
gdf

,geometry,meanHeight,RegionName,quadkey
0,"POLYGON ((-108.20774 33.19588, -108.20778 33.1...",2.908320,UnitedStates,23102312
1,"POLYGON ((-107.67342 33.33829, -107.67356 33.3...",-1.000000,UnitedStates,23102312
2,"POLYGON ((-107.59869 33.27274, -107.59853 33.2...",-1.000000,UnitedStates,23102312
3,"POLYGON ((-107.64454 33.34756, -107.64446 33.3...",-1.000000,UnitedStates,23102312
4,"POLYGON ((-107.64928 33.34947, -107.6493 33.34...",-1.000000,UnitedStates,23102312
...,...,...,...,...
25120,"POLYGON ((-104.22538 32.43169, -104.22538 32.4...",2.504073,UnitedStates,23103233
25121,"POLYGON ((-104.22057 32.38902, -104.22071 32.3...",-1.000000,UnitedStates,23103233
25122,"POLYGON ((-104.2181 32.38154, -104.21826 32.38...",3.599401,UnitedStates,23103233
25123,"POLYGON ((-104.18929 32.34827, -104.18929 32.3...",-1.000000,UnitedStates,23103233


In [56]:
buildings_in_perimiters = gpd.sjoin(gdf, perimeters, predicate='within')
buildings_in_perimiters

,geometry,meanHeight,RegionName,quadkey,index_right,duration,farea,fireid,flinelen,fperim,geom_counts,isactive,low_confidence_grouping,meanfrp,n_newpixels,n_pixels,pixden,primarykey,region,t
549,"POLYGON ((-108.09954 33.68163, -108.09958 33.6...",-1.000000,UnitedStates,23102312,52,4.0,46.295951,57474,0.000000,72.882442,None,0,0,45.843333,3,197,4.255232,CONUS|57474|2025-06-16T12:00:00,CONUS,2025-06-16T12:00:00
93,"POLYGON ((-108.16982 33.03085, -108.16967 33.0...",4.616078,UnitedStates,23102330,48,12.0,224.162856,57334,1.506331,68.090688,None,1,0,2.012500,4,1756,7.833590,CONUS|57334|2025-06-24T00:00:00,CONUS,2025-06-24T00:00:00
223,"POLYGON ((-108.14718 33.02737, -108.14718 33.0...",-1.000000,UnitedStates,23102330,48,12.0,224.162856,57334,1.506331,68.090688,None,1,0,2.012500,4,1756,7.833590,CONUS|57334|2025-06-24T00:00:00,CONUS,2025-06-24T00:00:00
381,"POLYGON ((-108.02127 32.94688, -108.02125 32.9...",3.737103,UnitedStates,23102330,48,12.0,224.162856,57334,1.506331,68.090688,None,1,0,2.012500,4,1756,7.833590,CONUS|57334|2025-06-24T00:00:00,CONUS,2025-06-24T00:00:00
473,"POLYGON ((-108.1463 33.02554, -108.14628 33.02...",4.098737,UnitedStates,23102330,48,12.0,224.162856,57334,1.506331,68.090688,None,1,0,2.012500,4,1756,7.833590,CONUS|57334|2025-06-24T00:00:00,CONUS,2025-06-24T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40535,"POLYGON ((-103.35454 32.71437, -103.35444 32.7...",14.025512,UnitedStates,23103321,7,37.0,0.379438,49357,2.293656,2.293656,None,0,0,0.890000,1,8,21.083798,CONUS|49357|2025-06-15T00:00:00,CONUS,2025-06-15T00:00:00
580,"POLYGON ((-108.9688 34.90846, -108.9687 34.908...",-1.000000,UnitedStates,23102121,81,0.5,1.469288,58917,2.894745,5.712908,None,1,0,60.866667,3,7,4.764211,CONUS|58917|2025-06-20T12:00:00,CONUS,2025-06-20T12:00:00
4504,"POLYGON ((-108.96888 34.90836, -108.96876 34.9...",-1.000000,UnitedStates,23102121,81,0.5,1.469288,58917,2.894745,5.712908,None,1,0,60.866667,3,7,4.764211,CONUS|58917|2025-06-20T12:00:00,CONUS,2025-06-20T12:00:00
1634,"POLYGON ((-104.11126 32.2683, -104.11178 32.26...",19.529743,UnitedStates,23103233,59,6.0,0.170982,57868,1.525828,1.525828,None,1,0,0.450000,1,2,11.697167,CONUS|57868|2025-06-21T00:00:00,CONUS,2025-06-21T00:00:00


## Population

In [101]:
pop_url = "https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km_UNadj/2020/USA/usa_pd_2020_1km_UNadj.tif" # From https://data.humdata.org/dataset/worldpop-population-density-for-united-states-of-america

In [102]:
# population = rs.open(pop_url)

---

Trying to do it in a fully STAC way

In [103]:
stac_url = "https://openveda.cloud/api/stac"

In [104]:
veda_catalog = Client.open(stac_url)
veda_catalog.title

'VEDA (Visualization, Exploration, and Data Analysis) STAC API'

In [105]:
snapshot_perimeter = veda_catalog.collection_search(
    q="fire"
)

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


In [106]:
snapshot_perimeter.matched()

19

In [107]:
for collection in snapshot_perimeter.collections():
    print(collection.id)

barc-thomasfire
caldor-fire-behavior
disalexi-etsuppression
caldor-fire-burn-severity
eis_fire_perimeter
campfire-albedo-wsa-diff
frp-max-thomasfire
campfire-lst-day-diff
campfire-lst-night-diff
campfire-nlcd
hls-bais2-v2
landsat-nighttime-thermal
campfire-ndvi-diff
lis-tvegsuppression
lis-etsuppression
la-fires-hrrr-wind
la-fires-planet
la-fires-frp
la-fires-maxar
